In [2]:
from utils import read_records, create_collection

import gensim
from gensim.utils import simple_preprocess
from gsdmm import MovieGroupProcess
from IPython.display import display
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
import spacy


collection = create_collection("covid-19")
tweets = [tweet for tweet in read_records(collection)]
df = pd.DataFrame(tweets)
df = df.set_index('_id')

df.head()

,created_at,text,withheld
_id,,,
1550877882213433350,2022-07-23T16:17:55.000Z,"Joe was cured from cancer, asthma, and probabl...",NaN
1550877888521523200,2022-07-23T16:17:57.000Z,When I wished for an #adventure on my trip to ...,NaN
1550877860533092353,2022-07-23T16:17:50.000Z,BREAKING: health agency chief declares monkeyp...,NaN
1550877778651791370,2022-07-23T16:17:31.000Z,the # 1 symptom of the COVID-19 novel coronavi...,NaN
1550877892611014657,2022-07-23T16:17:58.000Z,COVID isn't working for them anymore so now th...,NaN


In [7]:
# functions for preprocessing

# Punctuation removal + lowercasing + links, mentions, new lines, hashtags, etc
def clean_tweet(pandas_series):
    '''Returns a cleaned Pandas Series object with the following changes:
    1- Removal of hashtags, new lines, retweet mentions, links and punctuation;
    2- lowercasing.
    '''
    regex_transformations = [r'\n','(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)',
                             'RT', '@[A-Za-z0-9\.-_:]+', '[,\.!?:;/]']

    for regex in regex_transformations:
        pandas_series = pandas_series.map(lambda x: re.sub(regex, '', x))

    pandas_series = pandas_series.map(lambda x: x.lower())
    
    return pandas_series


# Tokenization
def tweet_to_words(tweets):
    for tweet in tweets:
        yield(gensim.utils.simple_preprocess(str(tweet), deacc=True))
        # deacc=True removes punctuation


# Bigram and Trigram models
def bigram_trigram(list_words):
    # build the models
    bigram = gensim.models.Phrases(list_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[list_words], threshold=100)
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

nltk.download('stopwords')

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Remove stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Make bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/devilgoncalo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# cast tweets to numpy array
docs = df.to_numpy()

# create dictionary of all words in all documents
dictionary = gensim.corpora.Dictionary(docs)

# filter extreme cases out of dictionary
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# create variable containing length of dictionary/vocab
vocab_length = len(dictionary)

# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

# initialize GSDMM
gsdmm = MovieGroupProcess(K=15, alpha=0.1, beta=0.3, n_iters=15)

# fit GSDMM model
y = gsdmm.fit(docs, vocab_length)

TypeError: decoding to str: need a bytes-like object, float found